In [ ]:
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing import image
import matplotlib.image as mpimg

In [ ]:
print(tf.__version__)

In [ ]:
os.listdir('dataset')

In [ ]:
base_dir = 'dataset'
train_dir = os.path.join(base_dir,'train')
val_dir = os.path.join(base_dir,'test')
print(train_dir)
print(val_dir)

In [ ]:
os.listdir('dataset/train')

In [ ]:
os.listdir('dataset/test')

In [ ]:
train_a_dir = os.path.join(train_dir, 'a')
train_a_names = os.listdir(train_a_dir)
train_b_dir = os.path.join(train_dir, 'b')
train_b_names = os.listdir(train_b_dir)
train_c_dir = os.path.join(train_dir, 'c')
train_c_names = os.listdir(train_c_dir)

In [ ]:
val_a_dir = os.path.join(val_dir, 'a')
val_b_dir = os.path.join(val_dir, 'b')
val_c_dir = os.path.join(val_dir, 'c')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    seed=123,
    target_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical')

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    color_mode='rgb',
    class_mode='categorical')

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.90 and logs.get('val_accuracy')>0.90):
      print("\nloss is low, cancelling training now")
      self.model.stop_training=True

In [ ]:
callbacks = myCallback()
tracking = model.fit(train_generator,
                    epochs=10, 
                    validation_data=val_generator,
                    callbacks=[callbacks])

In [ ]:
def show_plot_line(value, validation):
    plt.plot(value, color='blue')
    plt.plot(validation, color='orange')
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
acc = tracking.history['accuracy']
val_acc = tracking.history['val_accuracy']
show_plot_line(acc, val_acc)

In [ ]:
loss = tracking.history['loss']
val_loss = tracking.history['val_loss']
show_plot_line(loss, val_loss)

In [ ]:
model.save_weights('../server/aic_ml_service/model/final_relief.h5')

In [ ]:
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
rock_img_files = [os.path.join(train_rock_dir, f) for f in train_rock_names]
paper_img_files = [os.path.join(train_paper_dir, f) for f in train_paper_names]
scissors_img_files = [os.path.join(train_scissors_dir, f) for f in train_scissors_names]
img_path = random.choice(rock_img_files + paper_img_files + scissors_img_files)

img = load_img(img_path, target_size=(224, 224))  
x = img_to_array(img) 
x = x.reshape((1,) + x.shape)

x /= 255

successive_feature_maps = visualization_model.predict(x)

layer_names = [layer.name for layer in model.layers[1:]]

for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    n_features = feature_map.shape[-1]  
    size = feature_map.shape[1]
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')